In [ ]:
#create noisy datasets (noise on background /plant) for the original (raw) images
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

# Load environment variables
load_dotenv()

# Define paths from .env
base_dir = os.getenv("BASE_DIR")
csv_path = os.getenv("CSV_FILENAME")
original_img_dir = os.getenv("ORIGINAL_IMG_DIR")
segmented_img_dir = os.getenv("SEGMENTED_IMG_DIR")
results_dir = os.getenv("RESULTS_DIR")
harsh_noise_dir = os.getenv("ORIGINAL_NOISY_BASE_DIR")

# Create output directory
os.makedirs(harsh_noise_dir, exist_ok=True)

# Load the test dataset
df = pd.read_csv(csv_path)
test_df = df[df["train_test_set"] == "test"].copy()
print(f"Loaded test dataset with {len(test_df)} samples")

def add_harsh_mixed_background_noise(image, mask, intensity=0.9):
    """
    Add extremely harsh mixed noise to background regions only.

    """
    binary_mask = (mask > 0).astype(np.uint8)
    background_mask = (1 - binary_mask)
    
    noisy_image = image.astype(np.float64)
    
    # Create background-only noise
    bg_coords = np.where(background_mask == 1)
    if len(bg_coords[0]) == 0:  # No background pixels
        return image
    
    # Apply harsh noise only to background
    bg_pixels = noisy_image[bg_coords]
    
    # 1. Extreme multiplicative noise for background
    speckle = np.random.gamma(0.5, intensity * 0.6, bg_pixels.shape)
    bg_pixels = bg_pixels * speckle
    
    # 2. Very high Gaussian noise
    gaussian = np.random.normal(0, intensity * 60, bg_pixels.shape)
    bg_pixels = bg_pixels + gaussian
    
    # 3. Random impulse corruption
    impulse_mask = np.random.random(bg_pixels.shape) < (intensity * 0.4)
    random_vals = np.random.randint(0, 256, bg_pixels.shape)
    bg_pixels[impulse_mask] = random_vals[impulse_mask]
    
    # 4. Complete channel randomization for some background pixels
    if np.random.random() < intensity * 0.5:
        channel = np.random.randint(0, 3)
        random_channel_mask = np.random.random(len(bg_coords[0])) < 0.3
        bg_pixels[random_channel_mask, channel] = np.random.randint(0, 256, np.sum(random_channel_mask))
    
    # Put modified background back
    noisy_image[bg_coords] = np.clip(bg_pixels, 0, 255)
    
    return noisy_image.astype(np.uint8)

def add_harsh_mixed_plant_noise(image, mask, intensity=0.75):
    """
    Add harsh mixed noise to plant regions only.
    Designed to severely degrade plant features
    """
    binary_mask = (mask > 0).astype(np.uint8)
    
    noisy_image = image.astype(np.float64)
    
    # Create plant-only noise
    plant_coords = np.where(binary_mask == 1)
    if len(plant_coords[0]) == 0:  # No plant pixels
        return image
    
    # Apply harsh noise only to plant regions
    plant_pixels = noisy_image[plant_coords]
    
    # 1. Moderate multiplicative noise (don't completely destroy plant)
    speckle = np.random.gamma(1.2, intensity * 0.3, plant_pixels.shape)
    plant_pixels = plant_pixels * speckle
    
    # 2. High Gaussian noise
    gaussian = np.random.normal(0, intensity * 35, plant_pixels.shape)
    plant_pixels = plant_pixels + gaussian
    
    # 3. Selective impulse noise (less aggressive than background)
    impulse_mask = np.random.random(plant_pixels.shape) < (intensity * 0.2)
    random_vals = np.random.randint(0, 256, plant_pixels.shape)
    plant_pixels[impulse_mask] = random_vals[impulse_mask]
    
    # 4. Texture corruption (add random texture patterns)
    texture_noise = np.random.normal(0, intensity * 20, plant_pixels.shape)
    plant_pixels = plant_pixels + texture_noise
    
    # 5. Edge corruption (corrupt some edges specifically)
    edges = cv2.Canny(mask, 50, 150)
    edge_coords = np.where(edges > 0)
    if len(edge_coords[0]) > 0:
        edge_corruption = np.random.random(len(edge_coords[0])) < (intensity * 0.3)
        corrupt_edges = np.where(edge_corruption)[0]
        if len(corrupt_edges) > 0:
            edge_y = edge_coords[0][corrupt_edges]
            edge_x = edge_coords[1][corrupt_edges]
            for i in range(len(edge_y)):
                noisy_image[edge_y[i], edge_x[i]] = np.random.randint(0, 256, 3)
    
    # Put modified plant back
    noisy_image[plant_coords] = np.clip(plant_pixels, 0, 255)
    
    return noisy_image.astype(np.uint8)

# Create two subdirectories for different noise applications
background_noise_dir = os.path.join(harsh_noise_dir, "background_only")
plant_noise_dir = os.path.join(harsh_noise_dir, "plant_only")

for dir_path in [background_noise_dir, plant_noise_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Process images
for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Creating Harsh Mixed Noise Datasets"):
    code = row['code']
    orig_path = os.path.join(original_img_dir, f"{code}.jpg")
    seg_path = os.path.join(segmented_img_dir, f"{code}.jpg")
    
    try:
        # Load original image
        orig_image = cv2.imread(orig_path)
        if orig_image is None:
            print(f"Warning: Failed to load image {orig_path}, skipping.")
            continue
        orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
        
        # Load segmented image for selective noise
        seg_image = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)
        
        # 1. Background noise (if segmentation available)
        if seg_image is not None:
            harsh_bg_noisy = add_harsh_mixed_background_noise(orig_image, seg_image, intensity=0.9)
            Image.fromarray(harsh_bg_noisy).save(os.path.join(background_noise_dir, f"{code}.jpg"), quality=95)
            
            # 2. Plant noise
            harsh_plant_noisy = add_harsh_mixed_plant_noise(orig_image, seg_image, intensity=0.75)
            Image.fromarray(harsh_plant_noisy).save(os.path.join(plant_noise_dir, f"{code}.jpg"), quality=95)
        else:
            # If no segmentation, skip background and plant noise
            print(f"Warning: No segmentation for {code}, skipping background and plant noise.")
        
    except Exception as e:
        print(f"Error processing image {code}: {str(e)}")

print(f"Harsh Mixed Noise Datasets created at: {harsh_noise_dir}")
print(f"Background noise: {len(os.listdir(background_noise_dir))} images")
print(f"Plant noise: {len(os.listdir(plant_noise_dir))} images")
print("\nExtreme noise types applied:")
print("- Mixed Multiplicative + Additive Noise")
print("- High-variance Gaussian + Speckle + Impulse")
print("- Random block corruption + Channel corruption")
print("- Edge-specific corruption for plant regions")
print("- Intensity levels: Background(0.9), Plant(0.75)")
print("\nThis creates harsh noise conditions for robustness testing!")

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

# Load environment variables
load_dotenv()

# Define paths from .env
base_dir = os.getenv("BASE_DIR")
csv_path = os.getenv("CSV_FILENAME")
segmented_img_dir = os.getenv("SEGMENTED_IMG_DIR")
results_dir = os.getenv("RESULTS_DIR")
harsh_segmented_noise_dir = os.getenv("SEGMENTED_NOISY_BASE_DIR")

# Create output directory
os.makedirs(harsh_segmented_noise_dir, exist_ok=True)

# Load the test dataset
df = pd.read_csv(csv_path)
test_df = df[df["train_test_set"] == "test"].copy()
print(f"Loaded test dataset with {len(test_df)} samples")

def add_harsh_mixed_background_noise(image, mask, intensity=0.9):
    """
    Add extremely harsh mixed noise to background regions only.
    
    """
    binary_mask = (mask > 0).astype(np.uint8)
    background_mask = (1 - binary_mask)
    
    noisy_image = image.astype(np.float64)
    
    # Create background-only noise
    bg_coords = np.where(background_mask == 1)
    if len(bg_coords[0]) == 0:  # No background pixels
        return image
    
    # Apply harsh noise only to background
    bg_pixels = noisy_image[bg_coords]
    
    # 1. Extreme multiplicative noise for background
    speckle = np.random.gamma(0.5, intensity * 0.6, bg_pixels.shape)
    bg_pixels = bg_pixels * speckle
    
    # 2. Very high Gaussian noise
    gaussian = np.random.normal(0, intensity * 60, bg_pixels.shape)
    bg_pixels = bg_pixels + gaussian
    
    # 3. Random impulse corruption
    impulse_mask = np.random.random(bg_pixels.shape) < (intensity * 0.4)
    random_vals = np.random.randint(0, 256, bg_pixels.shape)
    bg_pixels[impulse_mask] = random_vals[impulse_mask]
    
    # 4. Complete channel randomization for some background pixels
    if np.random.random() < intensity * 0.5:
        channel = np.random.randint(0, 3)
        random_channel_mask = np.random.random(len(bg_coords[0])) < 0.3
        bg_pixels[random_channel_mask, channel] = np.random.randint(0, 256, np.sum(random_channel_mask))
    
    # Put modified background back
    noisy_image[bg_coords] = np.clip(bg_pixels, 0, 255)
    
    return noisy_image.astype(np.uint8)

def add_harsh_mixed_plant_noise(image, mask, intensity=0.75):
    """
    Add harsh mixed noise to plant regions only.
    Designed to severely degrade plant features 
    """
    binary_mask = (mask > 0).astype(np.uint8)
    
    noisy_image = image.astype(np.float64)
    
    # Create plant-only noise
    plant_coords = np.where(binary_mask == 1)
    if len(plant_coords[0]) == 0:  # No plant pixels
        return image
    
    # Apply harsh noise only to plant regions
    plant_pixels = noisy_image[plant_coords]
    
    # 1. Moderate multiplicative noise (don't completely destroy plant)
    speckle = np.random.gamma(1.2, intensity * 0.3, plant_pixels.shape)
    plant_pixels = plant_pixels * speckle
    
    # 2. High Gaussian noise
    gaussian = np.random.normal(0, intensity * 35, plant_pixels.shape)
    plant_pixels = plant_pixels + gaussian
    
    # 3. Selective impulse noise (less aggressive than background)
    impulse_mask = np.random.random(plant_pixels.shape) < (intensity * 0.2)
    random_vals = np.random.randint(0, 256, plant_pixels.shape)
    plant_pixels[impulse_mask] = random_vals[impulse_mask]
    
    # 4. Texture corruption (add random texture patterns)
    texture_noise = np.random.normal(0, intensity * 20, plant_pixels.shape)
    plant_pixels = plant_pixels + texture_noise
    
    # 5. Edge corruption (corrupt some edges specifically)
    edges = cv2.Canny(mask, 50, 150)
    edge_coords = np.where(edges > 0)
    if len(edge_coords[0]) > 0:
        edge_corruption = np.random.random(len(edge_coords[0])) < (intensity * 0.3)
        corrupt_edges = np.where(edge_corruption)[0]
        if len(corrupt_edges) > 0:
            edge_y = edge_coords[0][corrupt_edges]
            edge_x = edge_coords[1][corrupt_edges]
            for i in range(len(edge_y)):
                noisy_image[edge_y[i], edge_x[i]] = np.random.randint(0, 256, 3)
    
    # Put modified plant back
    noisy_image[plant_coords] = np.clip(plant_pixels, 0, 255)
    
    return noisy_image.astype(np.uint8)

# Create two subdirectories for different noise applications
background_noise_dir = os.path.join(harsh_segmented_noise_dir, "background_only")
plant_noise_dir = os.path.join(harsh_segmented_noise_dir, "plant_only")

for dir_path in [background_noise_dir, plant_noise_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Process images
for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Creating Harsh Mixed Noise Segmented Datasets"):
    code = row['code']
    seg_path = os.path.join(segmented_img_dir, f"{code}.jpg")
    
    try:
        # Load segmented image as RGB (this will be our source image for noise)
        seg_image_rgb = cv2.imread(seg_path)
        if seg_image_rgb is None:
            print(f"Warning: Failed to load segmented image {seg_path}, skipping.")
            continue
        seg_image_rgb = cv2.cvtColor(seg_image_rgb, cv2.COLOR_BGR2RGB)
        
        # Load segmented image as grayscale for mask creation
        seg_image_gray = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)
        
        # 1. Background noise on segmented image (if segmentation available)
        if seg_image_gray is not None:
            harsh_bg_noisy = add_harsh_mixed_background_noise(seg_image_rgb, seg_image_gray, intensity=0.9)
            Image.fromarray(harsh_bg_noisy).save(os.path.join(background_noise_dir, f"{code}.jpg"), quality=95)
            
            # 2. Plant noise on segmented image
            harsh_plant_noisy = add_harsh_mixed_plant_noise(seg_image_rgb, seg_image_gray, intensity=0.75)
            Image.fromarray(harsh_plant_noisy).save(os.path.join(plant_noise_dir, f"{code}.jpg"), quality=95)
        else:
            # If no grayscale version available, skip background and plant noise
            print(f"Warning: No segmentation for {code}, skipping background and plant noise.")
        
    except Exception as e:
        print(f"Error processing segmented image {code}: {str(e)}")

print(f"Harsh Mixed Noise Segmented Datasets created at: {harsh_segmented_noise_dir}")
print(f"Background noise: {len(os.listdir(background_noise_dir))} images")
print(f"Plant noise: {len(os.listdir(plant_noise_dir))} images")
print("\nExtreme noise types applied to SEGMENTED images:")
print("- Mixed Multiplicative + Additive Noise")
print("- High-variance Gaussian + Speckle + Impulse")
print("- Random block corruption + Channel corruption")
print("- Edge-specific corruption for plant regions")
print("- Intensity levels: Background(0.9), Plant(0.75)")
print("\nThis creates harsh noise conditions for robustness testing on segmented images!")